**Copyright 2021 Antoine SIMOULIN.**

Licensed under the Apache License, Version 2.0 (the "License");

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Loi de Zipf et pré-traitements de textes

Contenu du notebook :
* Premier contact avec méthodes de Scrapping
* Introduction aux expressions régulières et aux opérations de pré-traitements du texte
* Validation empirique de la loi de Zipf et sensibilisation à la distribution statistique des corpus

In [ ]:
%%capture

!pip install --upgrade beautifulsoup4

In [ ]:
from bs4 import BeautifulSoup          # Python parsing library
from collections import Counter
import nltk                            # NLP library
nltk.download('gutenberg')           # Run at first use
from nltk.corpus import gutenberg      
from nltk.probability import FreqDist  
import os 
import re                              # Regular Expression (Regex) in Python
import requests
import sys
from tqdm.auto import tqdm

# IPython automatically reload all changed code
%load_ext autoreload
%autoreload 2

# Inline Figures with matplotlib
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
!test -f plots.py || wget -q https://raw.githubusercontent.com/AntoineSimoulin/m2-data-sciences/master/src/plots.py .
!test -f split_sentences.py || wget -q https://raw.githubusercontent.com/AntoineSimoulin/m2-data-sciences/master/Cours%201%20-%20Mod%C3%A9lisation%20statistique%20du%20langage/solutions/split_sentences.py -P solutions
!test -f tokenize.py || wget -q https://raw.githubusercontent.com/AntoineSimoulin/m2-data-sciences/master/Cours%201%20-%20Mod%C3%A9lisation%20statistique%20du%20langage/solutions/tokenize.py -P solutions

sys.path.append('.')
from plots import plot_word_counter, plot_zipf

## Littrerature française

On cherche à s'assurer de la validité de la loi de Zipf en français. Pour cela on va construire un corpus avec des romans de la littérature française. Dans la démonstration, on propose d'utiliser des romans de Victor Hugo et Marcel Proust mais vous pouvez choisir les auteurs de votre choix.

On va récupérer les livres sur le site https://www.gutenberg.org/. Un projet qui rassemble des livres libres de droit. On récupère les tomes des Misérables et de A la recherche du temps perdu.

In [ ]:
book_links = [
    # A la recherche du temps perdu
    'https://www.gutenberg.org/files/2650/2650-h/2650-h.htm',     # Du côté de chez Swann
    'https://www.gutenberg.org/files/2998/2998-h/2998-h.htm',     # À l'ombre des jeunes filles en fleurs Partie 1
    'https://www.gutenberg.org/files/2999/2999-h/2999-h.htm',     # À l'ombre des jeunes filles en fleurs Partie 2
    'https://www.gutenberg.org/files/3000/3000-h/3000-h.htm',     # À l'ombre des jeunes filles en fleurs Partie 3
    'https://www.gutenberg.org/files/8946/8946-h/8946-h.htm',     # Le Côté de Guermantes Partie 1
    'https://www.gutenberg.org/files/12999/12999-h/12999-h.htm',  # Le Côté de Guermantes Partie 2
    'https://www.gutenberg.org/files/13743/13743-h/13743-h.htm',  # Le Côté de Guermantes Partie 3
    'https://www.gutenberg.org/files/15288/15288-h/15288-h.htm',  # Sodome et Gomorrhe Partie 1
    'https://www.gutenberg.org/files/15075/15075-h/15075-h.htm',  # Sodome et Gomorrhe Partie 2
    'https://www.gutenberg.org/files/60720/60720-h/60720-h.htm',  # La Prisonnière 
    # Les misérables
    'https://www.ebooksgratuits.com/html/hugo_les_miserables_fantine.html',
    'https://www.ebooksgratuits.com/html/hugo_les_miserables_cosette.html',
    'https://www.ebooksgratuits.com/html/hugo_les_miserables_marius.html',
    'https://www.ebooksgratuits.com/html/hugo_les_miserables_idylle_plumet_epopee_st_denis.html',
    'https://www.ebooksgratuits.com/html/hugo_les_miserables_idylle_plumet_epopee_st_denis.html',
    'https://www.ebooksgratuits.com/html/hugo_les_miserables_jean_valjean.html'
]

On récupère le texte au format HTML à l'aide de la librairie Beautiful Soup. La page HTML est organisée en paragraphes qui suivent le découpage du livre.

Attention, le scrapping n'est pas toujours autorisé, il est toujours primordial de s'assurer des licences et disposition légales concernant les données que l'on cherche à récupérer.

In [ ]:
%%time

paragraphes_all = []

for link in tqdm(book_links):
  page = requests.get(link) 
  soup = BeautifulSoup(page.content, 'html.parser')
      
  paragraphes = soup.select('p', class_='MsoNormal', style='')
  paragraphes = [p.get_text(strip=True) for p in paragraphes]
  paragraphes = [' '.join(p.split()) for p in paragraphes]
  paragraphes_all.extend(paragraphes)

In [ ]:
print("Downloaded {:,d} books for a total of {:,d} paragraphs.".format(len(book_links), len(paragraphes_all)))

In [ ]:
print(paragraphes_all[0])

<hr>

## Pré-traitements : les expressions régulières

Chaque paragraphe est consitué d'un unique bloc de texte. En NLP, les corpus sont généralement séparés en phrases et enregistrés dans un fichier ou on retrouve une phrase par ligne.

Les expressions régulières sont un outil très puissant. Elles permetent de rechercher des informations sous une forme standardisée. Dans l'exemple suivant, on peut cherche à extraire les numéros de téléphones. On cherche donc une suite de 10 chiffres avec éventuellement des séparateurs entre les chiffres. On peut également chercher les dates, les adresses, les montants, températures ou tout autre motif. 

In [ ]:
sample_text = """
    Bonjour mon numéro de téléphone est le 04.56.55.33.66
    Super le mien est 0392020302
    Génial, je vous donne également mon tel : 03-02-02-12-89 et celui du bureau : +33 (0) 5 33 19 33 09
    Est-ce que vous seriez disponible pour un rendez vous le 06/12/20 vers 13h ?
    Les tickets coutent 13€95.
"""

Les regex permettent de chercher des motifs dans le texte. Ces motifs sont décrits par des expressions standardisées très spécifiques. En python, on peut utiliser les regex à l'aide de la librairie `re`

Par exemple, on peut chercher un chiffre dans le texte:

In [ ]:
digit_pattern = re.compile('\d')

digits = re.findall(digit_pattern, sample_text)
print(digits)

On peut également chercher l'ensemble des motifs ou l'on retrouve deux chiffres qui se suivent

In [ ]:
digit_pattern = re.compile('\d{2}')

digits = re.findall(digit_pattern, sample_text)
print(digits)

Finalement on peut chercher l'ensemble des motifs ou on retrouve plusieurs chiffres qui se suivent

In [ ]:
digit_pattern = re.compile('\d+')
# + pour capter si le motif apparait entre 1 et une infinité de fois
# * pour capter si le motif apparait entre 0 et une infinité de fois

digits = re.findall(digit_pattern, sample_text)
print(digits)

On peut également chercher des groupes plus complexes, par exemple un numéro de téléphone

In [ ]:
telephone_pattern = re.compile('[\d\.\-\(\) +]{5,}')
# [] est l'équivalent de "ou" pour l'ensemble des motifs de la liste
# {5,} est un quantifier plus précis que * ou +. 
# Ici, on cherche ce motif au moins 5 fois. 
# {,5} serait au maximum 5 et {5} exactement 5 fois

telephone = re.findall(telephone_pattern, sample_text)
print(telephone)

La librairie `re` comprend d'autres méthodes que `re.search`. Par exemple la fonction de substitution `re.sub` qui permet de remplacer le motif par un autre. Un exemple ici poour procéder à une dé-anonymisation des données.

In [ ]:
sample_text_anonymized = re.sub(telephone_pattern, ' XX.XX.XX.XX.XX ', sample_text)
print(sample_text_anonymized)

On pourra s'appuyer sur l'outil : https://regex101.com/. Un autre excellent site pour s'entrainer aux expresions régulières : https://alf.nu/RegexGolf et une cheat sheet : https://cheatography.com/davechild/cheat-sheets/regular-expressions/


<hr>

**Exercice 1.** Séparer les paragraphes en phrases.

In [ ]:
def split_into_sentences(text):
    #TODO à compléter
    
    return sentences

In [ ]:
# %load solutions/split_sentences.py

<hr>

In [ ]:
bookcorpus = []

for p in paragraphes_all:
    sentences = split_into_sentences(p)
    bookcorpus.extend(sentences)
    
print("Extracted {:,d} sentences".format(len(bookcorpus)))

In [ ]:
# save corpus
!test -d ./data || mkdir ./data

with open(os.path.join('.', 'data', 'miserables_temps_perdu.txt'), 'w') as f:
    for s in bookcorpus:
        f.write(s + '\n')
        
print("Saved corpus.")

In [ ]:
!cat ./data/miserables_temps_perdu.txt | wc -l

In [ ]:
# load corpus
with open(os.path.join('data', 'miserables_temps_perdu.txt'), 'r') as f:
    sentences = f.readlines()
    
sentences = [l.strip() for l in sentences if l]

On va maintenant séparer le corpus en tokens.

<hr>

**Exercice 2.** Effectuer la tokenization du corpus et créer le dictionnaire de vocabulaire

In [ ]:
def tokenize(txt):
    
    #TODO à compléter
    
    return tokens

In [ ]:
# %load solutions/tokenize.py

<hr>

In [ ]:
tokenized_corpus = [tokenize(s) for s in sentences]

In [ ]:
n_tokens = sum([len(t) for t in tokenized_corpus])
print("Corpus contains {:,d} tokens.".format(n_tokens))

In [ ]:
tokenized_corpus[0]

In [ ]:
tokenized_corpus_flatten = [ll for l in tokenized_corpus for ll in l]

In [ ]:
assert len(tokenized_corpus_flatten) == n_tokens

In [ ]:
tokens_counter = Counter(tokenized_corpus_flatten)

In [ ]:
plot_word_counter(tokens_counter, 50)

In [ ]:
plot_zipf(tokens_counter)

# Jul

On compare la distribution des mots avec un autre auteur français : le rappeur [Jul](https://fr.wikipedia.org/wiki/Jul_(chanteur)) pour s'assurer que l'on retrouve une forme de distribution similaire. Les fréquences de mots ont été évaluées sur 521 chansons dont les paroles ont été scrappées sur le [AZLyrics](https://www.azlyrics.com/j/jul.html). Les données ont été traitées avec le même script que précédemment. On a directement sauvegardé les fréquences d'apparitions pour l'ensemble des mots du vocabulaire

In [ ]:
!test -f data/jul_freqs.txt || wget -q https://raw.githubusercontent.com/AntoineSimoulin/m2-data-sciences/master/Cours%201%20-%20Mod%C3%A9lisation%20statistique%20du%20langage/data/jul_freqs.txt -P data/

In [ ]:
tokens_counter = Counter()

with open(os.path.join('data', 'jul_freqs.txt'), 'r') as f:
    for line in f:
        k, v = line.strip().split('\t')
        tokens_counter[k] = int(v)

In [ ]:
tokens_counter.most_common(10)

In [ ]:
plot_word_counter(tokens_counter, 50)

In [ ]:
plot_zipf(tokens_counter)

# English Books

On compare avec la litératture anglaise en utilisant en particulier les oeuvres de Jane Austen, Shakespeare. Ces dernières sont accessibles directement en utilisant la librairie `NLTK`.

In [ ]:
fd = FreqDist()
n_words = 0
for text in gutenberg.fileids():
    for word in gutenberg.words(text):
        fd[word] += 1
        n_words += 1

ranks = []
freqs = []
for rank, word in enumerate(fd):
    ranks.append(rank+1)
    freqs.append(fd[word])

In [ ]:
plot_zipf(fd)

In [ ]:
plot_word_counter(fd, 50)